In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

train_dir = '00_input/train'
im_size = 100
coords_size = 28

In [2]:
def read_csv(filename):
    from numpy import array
    res = {}
    with open(filename) as fhandle:
        next(fhandle)
        for line in fhandle:
            parts = line.rstrip('\n').split(',')
            coords = array([float(x) for x in parts[1:]], dtype='float64')
            res[parts[0]] = coords
    return res

train_gt = read_csv(train_dir+'/gt.csv')

In [3]:
def parse(train_gt, train_img_dir, info=False):
    from skimage.data import imread
    from scipy.ndimage import zoom
    train_X = np.zeros((len(train_gt), im_size, im_size, 1))
    train_Y = np.zeros((len(train_gt), coords_size))
    for i, img_name in enumerate(train_gt):
        img = imread(train_img_dir+'/'+img_name, as_grey=True)
        train_Y[i] = train_gt[img_name]
        for j in range(1, coords_size, 2):
            train_Y[i][j] *= im_size/img.shape[0]
        for j in range(0, coords_size, 2):
            train_Y[i][j] *= im_size/img.shape[1]
        train_Y[i] = (train_Y[i] / 100) 
        img = zoom(img, [im_size/img.shape[0], im_size/img.shape[1]])
        img = (img / 255) 
        train_X[i,:,:,0] = img
        del(img)
        if info and (i+1)%100 == 0:
            print('Image: ', i+1, end='\r')
    return train_X, train_Y

In [4]:
from keras.models import Model
from keras import backend as K
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence

from keras.layers import (Input, concatenate, Conv2D, MaxPooling2D, 
                          UpSampling2D, Convolution2D, ZeroPadding2D, 
                          BatchNormalization, Activation, concatenate, 
                          Flatten, Dense, merge)
from keras.optimizers import rmsprop

def get_model():
    inputs = Input(shape=(im_size, im_size, 1))
    conv = Conv2D(filters=256, kernel_size=(3,3), padding='same')(inputs)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    conv = Conv2D(filters=128, kernel_size=(3,3), padding='same')(relu)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    maxpool = MaxPooling2D()(relu)
    
    conv = Conv2D(filters=64, kernel_size=(3,3), padding='same')(maxpool)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    conv = Conv2D(filters=64, kernel_size=(3,3), padding='same')(relu)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    maxpool = MaxPooling2D()(relu)
    
    conv = Conv2D(filters=32, kernel_size=(3,3), padding='same')(maxpool)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    flatten = Flatten()(relu)
    predictions = Dense(coords_size, activation='tanh')(flatten)
    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='rmsprop', loss='mean_squared_error')
    return model

Using TensorFlow backend.


In [8]:
def train_detector(
        train_gt, 
        train_img_dir, 
        fast_train=False, 
        model_func=None, 
        model_name='{epoch:d}_{val_loss:.2f}.hdf5'):
    train_X, train_Y = parse(train_gt, train_img_dir, True)
    if model_func == None:
        model = get_model()
    else:
        model = model_func()
        model_name += '_{epoch:d}_{val_loss:.2f}.hdf5'
    model.summary()
    checkpoint = ModelCheckpoint(
        model_name, 
        monitor='val_loss', 
        verbose=1, 
        save_best_only=True, 
        period=1,
        save_weights_only=False)
    if fast_train:
        epochs = 1
        model.fit(train_X, train_Y, epochs=epochs, batch_size=100, validation_split=0.1)
    else:
        epochs = 20
        try:
            model.fit(train_X, train_Y, epochs=epochs, batch_size=100, callbacks=[checkpoint], validation_split=0.1)
            #model.save('facepoints_model.hdf5')
        except KeyboardInterrupt:
            #model.save('facepoints_model.hdf5')
            print('Training interrupted')
    return model

In [6]:
train_detector(train_gt, train_dir+'/images')

/home/g5sokolovroman/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 256)     2560      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 256)     1024      
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 256)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 128)     295040    
_________________________________________________________________
batch_normalization_2 (Batch (None, 100, 100, 128)     512       
_________________________________________________________________
activation_2 (Activation)    (None, 100, 100, 128)     0         
__________

In [ ]:
def parse_test():
    

In [ ]:
def detect():